**데이터프레임 출력 파서(PandasDataFrameOutputParser)**

    - 구조화된 데이터를 다루기 위한 포괄적인 도구 세트
    - 데이터 정제, 변환 및 분석과 같은 작업에 다양하게 활용 가능
    - 사용자가 임의의 pandas dataframe을 지정하고 해당 dataframe에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있는 LLM을 요청하게 할 수 있게 해줌

In [1]:
from dotenv import load_dotenv

In [6]:
import pandas as pd
import pprint

from typing import Any, Dict

from langchain_openai import ChatOpenAI
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import PromptTemplate

In [4]:
model = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

In [7]:
# format_paser_output 함수로 파서 출력을 사전 형식으로 변환하고 예쁘게 출력
# 출력된 각 키에 대해, 해당 키의 값을 `.to_dict()` 메소드로 사전 형식으로 변환
# pprint.PrettyPrinter 클래스를 사용해서 출력을 위한 가독성 높임 

def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
        
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [11]:
import os

os.listdir('./data/')

['titanic.csv']

In [12]:
df = pd.read_csv('./data/titanic.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [20]:
# 사용자 쿼리를 처리하기 위한 프롬프트 설정
# PromptTemplate으로 쿼리 포맷과 지시사항 포함한 템플릿 생성
# parser.get_format_instruction() 함수를 호출해 포맷 지시사항 가져옴
# prompt, model, parser로 chain 생성
# chain.invoke로 df_query에 대한 처리 실행
# foramt_parser_output 함수로 parser의 출력 포맷

df_query = "Retrieve the passenges ages."
parser = PandasDataFrameOutputParser(dataframe=df)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

chain = prompt | model | parser

parser_output = chain.invoke({"query" : df_query})
parser_output

{'Age': 0     22.0
 1     38.0
 2     26.0
 3     35.0
 4     35.0
 5      NaN
 6     54.0
 7      2.0
 8     27.0
 9     14.0
 10     4.0
 11    58.0
 12    20.0
 13    39.0
 14    14.0
 15    55.0
 16     2.0
 17     NaN
 18    31.0
 19     NaN
 Name: Age, dtype: float64}

In [23]:
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [26]:
# 데이터 프레임의 첫 번째 행을 검색하는 쿼리

df_query = "Retrieve the first row"
parser = PandasDataFrameOutputParser(dataframe=df)

prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions}
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

parser_output

{'0': PassengerId                          1
 Survived                             0
 Pclass                               3
 Name           Braund, Mr. Owen Harris
 Sex                               male
 Age                               22.0
 SibSp                                1
 Parch                                0
 Ticket                       A/5 21171
 Fare                              7.25
 Cabin                              NaN
 Embarked                             S
 Name: 0, dtype: object}

In [27]:
format_parser_output(parser_output)

{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


In [28]:
#pandas DataFrame에서 특정 열의 일부 행의 평균을 검색하는 작업

df['Age'].head().mean()

31.2

In [32]:
df_query = "Retrieve the average of the ages from row 0 to 4."

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

print(parser_output)

{'mean': 31.2}


In [37]:
df_query = "Retrieve the average Fare."

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

In [34]:
parser_output

{'mean': 22.19937}

In [36]:
df['Fare'].mean()

22.19937